In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Extraction_Data")\
        .getOrCreate()

24/10/05 07:27:18 WARN Utils: Your hostname, Ubuntu-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/05 07:27:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/05 07:27:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/10/05 07:27:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [13]:
from datetime import datetime

current_year = datetime.now().year

In [15]:
type(datetime.now().year)

int

In [14]:
ano_inicial = current_year
ano_final = current_year

dif_anos = (ano_final - ano_inicial) + 1
for i in range(dif_anos):
    print(ano_inicial + i)

2024


In [5]:
origin = '/home/adriano/Documentos/airflow/database/dest/bronze/execucao-financeira'
process_all = False
ano_final = current_year
ano_inicial = current_year


if process_all == False:
    origin_file = origin + '/' + str(ano_inicial) + '_' + str(ano_final) + '.json'
    df = spark.read.json(origin_file)

print(df.printSchema())
df.show()

root
 |-- autorEmenda: string (nullable = true)
 |-- codigoAmparoLegal: long (nullable = true)
 |-- descricaoEmpenho: string (nullable = true)
 |-- fonteRecurso: string (nullable = true)
 |-- idProjetoInvestimento: string (nullable = true)
 |-- informacoesComplementares: string (nullable = true)
 |-- localEntrega: string (nullable = true)
 |-- naturezaDespesa: string (nullable = true)
 |-- nomeEsferaOrcamentaria: string (nullable = true)
 |-- nomeFavorecido: string (nullable = true)
 |-- nomeTipoEmpenho: string (nullable = true)
 |-- nrPtres: string (nullable = true)
 |-- numeroNotaEmpenhoGerada: string (nullable = true)
 |-- numeroProcesso: string (nullable = true)
 |-- pagina: long (nullable = true)
 |-- planoInterno: string (nullable = true)
 |-- planoOrcamentario: string (nullable = true)
 |-- resultadoPrimario: string (nullable = true)
 |-- tamanhoDaPagina: long (nullable = true)
 |-- tipoCredito: string (nullable = true)
 |-- ugEmitente: string (nullable = true)
 |-- ugResponsave

+------------+-----------------+--------------------+------------+---------------------+-------------------------+--------------------+---------------+----------------------+--------------------+---------------+-----------------+-----------------------+--------------------+------+------------+-----------------+-----------------+---------------+-----------+----------+-------------+-------------------+------------+
| autorEmenda|codigoAmparoLegal|    descricaoEmpenho|fonteRecurso|idProjetoInvestimento|informacoesComplementares|        localEntrega|naturezaDespesa|nomeEsferaOrcamentaria|      nomeFavorecido|nomeTipoEmpenho|          nrPtres|numeroNotaEmpenhoGerada|      numeroProcesso|pagina|planoInterno|planoOrcamentario|resultadoPrimario|tamanhoDaPagina|tipoCredito|ugEmitente|ugResponsavel|unidadeOrcamentaria|valorEmpenho|
+------------+-----------------+--------------------+------------+---------------------+-------------------------+--------------------+---------------+---------------